In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from pathlib import Path

pd.set_option("display.max_columns", 500)

# Creating Top10 HCPCS CMS 2017 PUF file

In [2]:
CY2017 = pd.read_csv("./data/Medicare_Provider_Util_Payment_PUF_CY2017.txt", sep="\t",header=0,skiprows=[1])

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = CY2017.copy()

In [4]:
df.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_provider_mi', 'nppes_credentials', 'nppes_provider_gender',
       'nppes_entity_code', 'nppes_provider_street1', 'nppes_provider_street2',
       'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',
       'nppes_provider_country', 'provider_type',
       'medicare_participation_indicator', 'place_of_service', 'hcpcs_code',
       'hcpcs_description', 'hcpcs_drug_indicator', 'line_srvc_cnt',
       'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt',
       'average_Medicare_standard_amt'],
      dtype='object')

In [56]:
df['Total_submitted_chrg_amt'] = df['line_srvc_cnt'] * df['average_submitted_chrg_amt']
df['Total_Medicare_payment_amt'] = df['line_srvc_cnt'] * df['average_Medicare_payment_amt']

In [57]:
df.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt
6,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00
7,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99232,"Subsequent hospital inpatient care, typically ...",N,627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94
8,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99233,"Subsequent hospital inpatient care, typically ...",N,207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88
10,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99291,Critical care delivery critically ill or injur...,N,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00
39,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,99213,Established patient office or other outpatient...,N,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98


In [8]:
#per-row (per HCPS not grouped) --(wrong code)
#top = df.sort_values(by='Total_submitted_chrg_amt', ascending=False).hcpcs_code.unique() 
#x = df[df.hcpcs_code.isin(top[0:10])]
#x.shape

In [9]:
#x.hcpcs_code.value_counts()

In [29]:
Total_submitted = df.Total_submitted_chrg_amt.sum()

In [47]:
# sort by top 10 hcpcs:
top_hcpcs = df.groupby(['hcpcs_code'])['Total_submitted_chrg_amt'].sum().sort_values(ascending=False)
print(top_hcpcs[0:10]) #Top-10 hcpcs code Total_submitted_chrg_amt
top10_hcpcs = top_hcpcs[0:10].index
top10_hcpcs

hcpcs_code
99214    2.032672e+10
99213    1.287074e+10
99285    1.244005e+10
66984    8.510616e+09
99232    7.573940e+09
99233    5.667375e+09
A0427    5.432417e+09
99223    4.681324e+09
99291    4.499186e+09
99284    3.748309e+09
Name: Total_submitted_chrg_amt, dtype: float64


Index(['99214', '99213', '99285', '66984', '99232', '99233', 'A0427', '99223',
       '99291', '99284'],
      dtype='object', name='hcpcs_code')

In [58]:
df_top10 = df[df.hcpcs_code.isin(top10_hcpcs)]
df_top10.shape

(1520985, 28)

In [20]:
df_top10.hcpcs_code.value_counts() #number of submitted charges
# bene_unique_cnt / total_bene_unique_cnt
# unique_npi / total_unique_npi

99213    457843
99214    428268
99232    175336
99223    133368
99233    120664
99291     62680
99284     60805
99285     57220
66984     17613
A0427      7188
Name: hcpcs_code, dtype: int64

In [30]:
Total_bene_unq = df.bene_unique_cnt.sum()
Total_line_srvc = df.line_srvc_cnt.sum()

In [26]:
df_bene = df_top10.groupby('hcpcs_code')['bene_unique_cnt'].sum().sort_values(ascending=False)
df_line = df_top10.groupby('hcpcs_code')['line_srvc_cnt'].sum().sort_values(ascending=False)

In [46]:
# Making a new summary DataFrame:
summary_top10hcpcs = pd.DataFrame({'Total_submitted_percent': ((top_hcpcs[0:10] / Total_submitted) * 100),
              'Total_beneficiary_percent': ((df_bene / Total_bene_unq) * 100),
              'Total_line_service_percent': ((df_line / Total_line_srvc) * 100)})
summary_top10hcpcs.index.name="Top10_hcpcs"
summary_top10hcpcs.sort_values(by='Total_submitted_percent', ascending=False)

,Total_submitted_percent,Total_beneficiary_percent,Total_line_service_percent
Top10_hcpcs,,,
99214,6.438335,6.345879,4.194636
99213,4.076711,6.413324,4.001411
99285,3.940294,1.294201,0.485196
66984,2.695674,0.237566,0.317766
99232,2.398989,1.930589,1.936963
99233,1.795099,1.112096,0.935694
A0427,1.720677,0.408946,0.209624
99223,1.482774,1.111840,0.428854
99291,1.425083,0.388740,0.223256


In [48]:
df_top10.groupby('hcpcs_code')['npi'].count()

hcpcs_code
66984     17613
99213    457843
99214    428268
99223    133368
99232    175336
99233    120664
99284     60805
99285     57220
99291     62680
A0427      7188
Name: npi, dtype: int64

In [59]:
df_top10.shape

(1520985, 28)

In [50]:
df_top10.hcpcs_code.value_counts()

99213    457843
99214    428268
99232    175336
99223    133368
99233    120664
99291     62680
99284     60805
99285     57220
66984     17613
A0427      7188
Name: hcpcs_code, dtype: int64

In [51]:
#df_top10.to_csv(Path("data","top10_hcpcs.csv")) #create a new csv file

In [52]:
#sort via highest avg Total_average_submitted_charge_amount
#pick top 10 
#find proportion for the top 10
# drop all columns not in top 10

# LEIE and TOP10 HCPCS df CleanUP

In [53]:
#LEIE
LEIE = pd.read_csv('data/LEIE_08_2019.csv')

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
#Top10-HCPCS
df = pd.read_csv('data/top10_hcpcs.csv', 
                 dtype={'hcpcs_code':object},
                 index_col=0)

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/rad/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [55]:
df.shape

(1520985, 28)

In [60]:
df.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt
6,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00
7,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99232,"Subsequent hospital inpatient care, typically ...",N,627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94
8,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99233,"Subsequent hospital inpatient care, typically ...",N,207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88
10,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99291,Critical care delivery critically ill or injur...,N,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00
39,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,99213,Established patient office or other outpatient...,N,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98


In [61]:
df.hcpcs_code.nunique()

10

In [62]:
df.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_provider_mi', 'nppes_credentials', 'nppes_provider_gender',
       'nppes_entity_code', 'nppes_provider_street1', 'nppes_provider_street2',
       'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',
       'nppes_provider_country', 'provider_type',
       'medicare_participation_indicator', 'place_of_service', 'hcpcs_code',
       'hcpcs_description', 'hcpcs_drug_indicator', 'line_srvc_cnt',
       'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt',
       'average_Medicare_standard_amt', 'Total_submitted_chrg_amt',
       'Total_Medicare_payment_amt'],
      dtype='object')

#### Filter df based on:
- df.nppes_provider_country == 'US'
- df.medicare_participation_indicator == 'Y'
- df.nppes_entity_code=='I'

In [63]:
# need to drop this!
((df.nppes_provider_country != 'US') | (df.medicare_participation_indicator == 'N') |
 (df.nppes_entity_code=='O')).sum()

9458

In [64]:
# Fix all zip to 5 digits and convert to string
df['nppes_provider_zip'] = df.nppes_provider_zip.astype(str)
df['nppes_provider_zip'] = df['nppes_provider_zip'].str[0:5]

In [65]:
# FILTER DF
df = df.loc[(df.nppes_provider_country == 'US') & (df.medicare_participation_indicator == 'Y')\
            & (df.nppes_entity_code=='I')]
dropcols = ["nppes_provider_country", "medicare_participation_indicator", \
            "nppes_entity_code", "nppes_provider_street2", \
            "nppes_provider_mi", "hcpcs_drug_indicator"]

In [66]:
df = df.drop(dropcols, axis=1) #drop unnecessary columns

In [67]:
df.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_credentials', 'nppes_provider_gender', 'nppes_provider_street1',
       'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',
       'provider_type', 'place_of_service', 'hcpcs_code', 'hcpcs_description',
       'line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt',
       'average_Medicare_allowed_amt', 'average_submitted_chrg_amt',
       'average_Medicare_payment_amt', 'average_Medicare_standard_amt',
       'Total_submitted_chrg_amt', 'Total_Medicare_payment_amt'],
      dtype='object')

In [68]:
# RENAME to proper column names:
df.columns = ["npi", "lastname", "firstname", "credentials", "gender", "address", "city",\
              "zip", "state", "provider_type", "place_of_service", "hcpcs_code", "hcpcs_description", \
              "line_srvc_cnt", "bene_unique_cnt", "bene_day_srvc_cnt", \
              "average_Medicare_allowed_amt", "average_submitted_chrg_amt", \
              "average_Medicare_payment_amt", "average_Medicare_standard_amt",\
              "Total_submitted_chrg_amt", "Total_Medicare_payment_amt"]

In [69]:
# HCPCS counts have decreased after the filtering!!
df.hcpcs_code.value_counts()

99213    457650
99214    428097
99232    175317
99223    133359
99233    120656
99291     62675
99284     60796
99285     57211
66984     15761
A0427         5
Name: hcpcs_code, dtype: int64

In [70]:
df.shape #shape has changed!

(1511527, 22)

In [71]:
df.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt
6,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00
7,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94
8,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88
10,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00
39,1003000142,KHALIL,RASHID,M.D.,M,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98


In [72]:
LEIE.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
0,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN
1,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN
2,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN
3,NaN,NaN,,1951 FLATBUSH AVENUE PHARMACY,PHARMACY,NaN,NaN,0,NaN,1951 FLATBUSH AVE,BROOKLYN,NY,11234,1128b5,20090319,0,0,NaN
4,NaN,NaN,,"1ST COMMUNITY HEALTH CTR, LTD",CLINIC,NaN,NaN,0,NaN,3138 W CERMAK ROAD,CHICAGO,IL,60623,1128a1,19940524,0,0,NaN


In [73]:
LEIE.columns = LEIE.columns.str.lower()

In [74]:
# need to remove this from LEIE!
(LEIE.busname.notnull()).sum()

3108

In [75]:
# Subset LEIE on only indiv and work only on this file
LEIE_indiv = LEIE[LEIE.busname.isnull()]
LEIE_indiv.shape 

(69368, 18)

# Matching npi

In [76]:
# Finding duplicates in the LEIE file -- there are 101 duplicated npis!! 
x = LEIE[LEIE['npi']!=0]
x[x.npi.duplicated()].shape #101 duplicated npis

(101, 18)

In [77]:
LEIE[LEIE.npi!=0].shape[0] #5269 npis not zero

5269

In [78]:
LEIE.npi.nunique() #5169 unique npi (total 100 are duplicated!)

5169

In [79]:
#drop all duplicated npi
LEIE_uniquenpi = LEIE.drop_duplicates(subset='npi') #subset on npi column
LEIE_uniquenpi.shape

(5169, 18)

In [80]:
npi_same = pd.merge(df, LEIE_uniquenpi, how='inner',on='npi').npi.values

In [81]:
len(npi_same)

277

In [82]:
df.npi.isin(npi_same).value_counts() #count

False    1511250
True         277
Name: npi, dtype: int64

In [83]:
#only 277 npi's matched!
df[df.npi.isin(npi_same)].shape

(277, 22)

In [84]:
df[df.npi.isin(npi_same)]['npi'].nunique() #unique npi matches from LEIE and df

137

In [85]:
(df.npi.isin(npi_same).astype(int)==1).sum() #converting bool to int

277

In [86]:
#This step will be done later!! 
#df['Target'] = df.npi.isin(npi_same).astype(int) 

In [87]:
df.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt
6,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00
7,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94
8,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88
10,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00
39,1003000142,KHALIL,RASHID,M.D.,M,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98


# Merging Aparna's Files:

In [88]:
# LEIE filtered by NPI==0 and BUSNAME==NULL 
leie_corrected = pd.read_csv('data/Aparna/LEIE_reduced.csv', index_col=0)

In [89]:
# CMS2017 file - subsetted based on "isin" using "zip"->"lastname"->"address"
CMS_subset = pd.read_csv('data/Aparna/LEIE_labels_1.csv', index_col=0)

In [90]:
CMS_subset.hcpcs_code.nunique()

9

In [91]:
leie_corrected.shape

(64432, 21)

In [92]:
colskeep = ["lastname", "npi_leie", "zip_new", "street1", "EXCLTYPE", "EXCLDATE"]

In [93]:
leie_corrected = leie_corrected.filter(items=colskeep, axis=1)

In [94]:
leie_corrected.shape

(64432, 6)

In [95]:
leie_corrected.head()

,lastname,npi_leie,zip_new,street1,EXCLTYPE,EXCLDATE
3108,aalders,0,60453,9527 south kenton,1128b4,20030820
3109,aaltonen,0,99201,32 w pacific ave,1128a4,20120419
3110,aamir,0,48302,3956 columbia court,1128a1,20170220
3111,aaron,0,85018,3325 n rose circle drive,1128b4,20171019
3112,aaron,0,35504,5911 old pineywoods rd,1128b4,20061120


In [96]:
leie_corrected['year'] = pd.to_datetime(leie_corrected["EXCLDATE"], format="%Y%m%d").dt.year

In [97]:
leie_corrected.head()

,lastname,npi_leie,zip_new,street1,EXCLTYPE,EXCLDATE,year
3108,aalders,0,60453,9527 south kenton,1128b4,20030820,2003
3109,aaltonen,0,99201,32 w pacific ave,1128a4,20120419,2012
3110,aamir,0,48302,3956 columbia court,1128a1,20170220,2017
3111,aaron,0,85018,3325 n rose circle drive,1128b4,20171019,2017
3112,aaron,0,35504,5911 old pineywoods rd,1128b4,20061120,2006


In [98]:
cmscolskeep = ["npi", "lastname", "nppes_provider_first_name", "zip_new","street1"]

In [99]:
CMS_subset = CMS_subset.filter(items=cmscolskeep, axis=1)

In [100]:
CMS_subset.head()

,npi,lastname,nppes_provider_first_name,zip_new,street1
7901,1003044611,danek,SAMANTHA,49201,205 n east ave
7902,1003044611,danek,SAMANTHA,49201,205 n east ave
19615,1003125485,hanna,EHAB,11706,301 e main st
25144,1003184599,eastman,MEGAN,50501,800 kenyon rd
25145,1003184599,eastman,MEGAN,50501,800 kenyon rd


In [101]:
CMS_subset.shape

(3915, 5)

In [102]:
merged_file = pd.merge(CMS_subset, leie_corrected, how="inner", left_on=["lastname", "zip_new", "street1"],\
         right_on=["lastname","zip_new", "street1"])

In [104]:
merged_file #total 8 matches in the LEIE.npi==0 and top10hcpcs_CMSfile

,npi,lastname,nppes_provider_first_name,zip_new,street1,npi_leie,EXCLTYPE,EXCLDATE,year
0,1285969980,siddiqui,IQBAL,7936,9 tower dr,0,1128a1,20010719,2001
1,1285969980,siddiqui,IQBAL,7936,9 tower dr,0,1128a1,20010719,2001
2,1285969980,siddiqui,IQBAL,7936,9 tower dr,0,1128a1,20010719,2001
3,1447436209,fisher,EMILY,77030,6560 fannin st,0,1128b4,19981020,1998
4,1447436209,fisher,EMILY,77030,6560 fannin st,0,1128b4,19981020,1998
5,1518920602,lee,VICTOR,95123,5710 cahalan ave,0,1128a1,19830107,1983
6,1891090262,antoon,PATRICK,71860,218 church st,0,1128a1,20061120,2006
7,1891090262,antoon,PATRICK,71860,218 church st,0,1128a1,20061120,2006


## Adding EXCLTYPE and year

In [105]:
#LEIE
LEIE = pd.read_csv('data/LEIE_08_2019.csv')

In [106]:
LEIE.columns = LEIE.columns.str.lower()

In [107]:
LEIE_indiv = LEIE[LEIE.busname.isnull()]
LEIE_indiv.shape

(69368, 18)

In [108]:
LEIE_indiv['year'] = pd.to_datetime(LEIE_indiv["excldate"], format="%Y%m%d").dt.year

/Users/rad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [109]:
LEIE_indiv.head()

,lastname,firstname,midname,busname,general,specialty,upin,npi,dob,address,city,state,zip,excltype,excldate,reindate,waiverdate,wvrstate,year
3108,AALDERS,NANCY,BLOCK,NaN,HOSPITAL,NURSE/NURSES AIDE,NaN,0,19640305.0,9527 SOUTH KENTON,OAK LAWN,IL,60453,1128b4,20030820,0,0,NaN,2003
3109,AALTONEN,NICKOLAS,A,NaN,IND- LIC HC SERV PRO,PHARMACY TECHNICIAN,NaN,0,19880123.0,32 W PACIFIC AVE,SPOKANE,WA,99201,1128a4,20120419,0,0,NaN,2012
3110,AAMIR,MUHAMMAD,SOHAIL,NaN,BUS OWNER/EXEC,HOME HEALTH AGENCY,NaN,0,19700910.0,3956 COLUMBIA COURT,BLOOMFIELD HILLS,MI,48302,1128a1,20170220,0,0,NaN,2017
3111,AARON,ALINA,A,NaN,IND- LIC HC SERV PRO,NURSE/NURSES AIDE,NaN,0,19870209.0,3325 N ROSE CIRCLE DRIVE,PHOENIX,AZ,85018,1128b4,20171019,0,0,NaN,2017
3112,AARON,BARBARA,KAY,NaN,NURSING PROFESSION,NURSE/NURSES AIDE,NaN,0,19650914.0,5911 OLD PINEYWOODS RD,JASPER,AL,35504,1128b4,20061120,0,0,NaN,2006


In [110]:
merged_file

,npi,lastname,nppes_provider_first_name,zip_new,street1,npi_leie,EXCLTYPE,EXCLDATE,year
0,1285969980,siddiqui,IQBAL,7936,9 tower dr,0,1128a1,20010719,2001
1,1285969980,siddiqui,IQBAL,7936,9 tower dr,0,1128a1,20010719,2001
2,1285969980,siddiqui,IQBAL,7936,9 tower dr,0,1128a1,20010719,2001
3,1447436209,fisher,EMILY,77030,6560 fannin st,0,1128b4,19981020,1998
4,1447436209,fisher,EMILY,77030,6560 fannin st,0,1128b4,19981020,1998
5,1518920602,lee,VICTOR,95123,5710 cahalan ave,0,1128a1,19830107,1983
6,1891090262,antoon,PATRICK,71860,218 church st,0,1128a1,20061120,2006
7,1891090262,antoon,PATRICK,71860,218 church st,0,1128a1,20061120,2006


In [111]:
#dropping unnecessary cols from merged file (total 8 rows)
merged_file = merged_file.drop(['lastname', 'nppes_provider_first_name', 'zip_new', 'street1','npi_leie','EXCLDATE'], axis=1)
merged_file

,npi,EXCLTYPE,year
0,1285969980,1128a1,2001
1,1285969980,1128a1,2001
2,1285969980,1128a1,2001
3,1447436209,1128b4,1998
4,1447436209,1128b4,1998
5,1518920602,1128a1,1983
6,1891090262,1128a1,2006
7,1891090262,1128a1,2006


In [112]:
# dropping unnecessary cols from LEIE_indiv file
keepcols1 = ['npi', 'excltype', 'year']
LEIE_x = LEIE_indiv[LEIE_indiv.npi!=0].filter(items=keepcols1)
LEIE_x.columns = ['npi', 'EXCLTYPE', 'year']
LEIE_x.head()

,npi,EXCLTYPE,year
3123,1477537496,1128b4,2014
3142,1194807255,1128b4,2018
3143,1225029184,1128a2,2014
3212,1487870507,1128a4,2018
3216,1497811368,1128a1,1999


In [113]:
print(merged_file.shape)
print(LEIE_x.shape)

(8, 3)
(4934, 3)


In [114]:
# concat the two files - overall LEIE file with NPIs !!
LEIE_npi_match = pd.concat([merged_file, LEIE_x], sort=False)
LEIE_npi_match.head()

,npi,EXCLTYPE,year
0,1285969980,1128a1,2001
1,1285969980,1128a1,2001
2,1285969980,1128a1,2001
3,1447436209,1128b4,1998
4,1447436209,1128b4,1998


In [115]:
LEIE_npi_match.shape

(4942, 3)

In [116]:
df.shape 

(1511527, 22)

In [117]:
# MERGE THE TWO FILES -- ADD EXCLTYPE AND YEAR
# create new DataFrame called "combo"
combo = pd.merge(df, LEIE_npi_match, how="left", on="npi")
combo.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,EXCLTYPE,year
0,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,NaN,NaN
1,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,NaN,NaN
2,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,NaN,NaN
3,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,NaN,NaN
4,1003000142,KHALIL,RASHID,M.D.,M,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,NaN,NaN


In [118]:
combo.shape #note: df.shape is (1511527, 22) which is less than "LEFT" merge

(1511539, 24)

In [122]:
combo = combo.drop_duplicates()

In [123]:
combo.shape

(1511529, 24)

In [124]:
combo.EXCLTYPE.notnull().sum()

287

In [127]:
def imputeNone(df_input):
    df = df_input.copy()
    cols = ['EXCLTYPE', 'year']
    for col in cols:
        df[col] = np.where(df[col].isnull(), "None", df[col])
    return(df)

In [128]:
combo = imputeNone(combo) #impute "None" for NaN EXCLTYPE and year

In [129]:
combo.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,EXCLTYPE,year
0,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,None,None
1,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,None,None
2,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,None,None
3,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,None,None
4,1003000142,KHALIL,RASHID,M.D.,M,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,None,None


## Adding Target Column

In [131]:
# NOTE: npi_same are the matching NPIs between LEIE and df
len(npi_same) 

277

In [132]:
npi_same_list = npi_same.tolist() #convert to a list

In [133]:
# 8 more NPI we matched when LEIE(npi==0) and df
npi_more = merged_file.npi.tolist()
npi_more

[1285969980,
 1285969980,
 1285969980,
 1447436209,
 1447436209,
 1518920602,
 1891090262,
 1891090262]

In [134]:
# append 8 more npi to the list
for i in npi_more:
    npi_same_list.append(i)

In [135]:
# Total npi matched between LEIE and df
len(npi_same_list) 

285

In [136]:
(combo.npi.isin(npi_same_list)).sum()

287

In [137]:
# ADD TARGET COLUMN into the DataFrame combo
combo['Target'] = (combo.npi.isin(npi_same_list)).astype(int)

In [140]:
print("Total matched npi:", (combo['Target']==1).sum())
print("Total matched unique npi:", combo[combo.Target==1]['npi'].nunique())

Total matched npi: 287
Total matched unique npi: 141


In [142]:
# LOWER all the elements in the following columns below: 
cols = ['lastname', 'firstname', 'address', 'city']
for col in cols:
    combo[col] = combo[col].str.lower()

In [143]:
combo.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,EXCLTYPE,year,Target
0,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,None,None,0
1,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,None,None,0
2,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,None,None,0
3,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,None,None,0
4,1003000142,khalil,rashid,M.D.,M,4126 n holland sylvania rd,toledo,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,None,None,0


In [144]:
combo.shape #df.shape = (1511527, 22)

(1511529, 25)

In [148]:
combo.hcpcs_code.value_counts() #Should be 10 (top10hcpcs from df file)

99213    457651
99214    428098
99232    175317
99223    133359
99233    120656
99291     62675
99284     60796
99285     57211
66984     15761
A0427         5
Name: hcpcs_code, dtype: int64

In [151]:
combo.hcpcs_code.nunique()

10

In [147]:
#combo.to_csv("data/combo.csv",index_label=False) #write to csv

In [153]:
combo.shape

(1511529, 25)

# CLEANING THE COMBO FILE 

In [392]:
## Re-read the combo file that was just created!!
dfcombo = pd.read_csv("data/combo.csv",
                 dtype={'npi':object,
                        'lastname':object,
                        'firstname':object,
                        'credentials':object,
                        'gender':object,
                        'address':object,
                        'city':object,
                        'zip':object,
                        'state':object,
                        'provider_type':object,
                        'place_of_service':object,
                        'hcpcs_code':object,
                        'hcpcs_description':object,
                        'EXCLTYPE':object,
                        'year':object}) 

In [393]:
dfcombo.columns

Index(['npi', 'lastname', 'firstname', 'credentials', 'gender', 'address',
       'city', 'zip', 'state', 'provider_type', 'place_of_service',
       'hcpcs_code', 'hcpcs_description', 'line_srvc_cnt', 'bene_unique_cnt',
       'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt',
       'average_Medicare_standard_amt', 'Total_submitted_chrg_amt',
       'Total_Medicare_payment_amt', 'EXCLTYPE', 'year', 'Target'],
      dtype='object')

In [395]:
dfcombo.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,EXCLTYPE,year,Target
0,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,None,None,0
1,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,None,None,0
2,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,None,None,0
3,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,None,None,0
4,1003000142,khalil,rashid,M.D.,M,4126 n holland sylvania rd,toledo,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,None,None,0


In [396]:
#save Target (just incase if needed in future)
target = dfcombo.iloc[:,-1]

In [397]:
dfcombo.shape

(1511529, 25)

### Feature Engineering

In [398]:
dfcombo['Total_Medicare_std_payment_amt'] = dfcombo.line_srvc_cnt * dfcombo.average_Medicare_standard_amt

In [399]:
dfcombo['Total_Medicare_allowed_amt'] = dfcombo.line_srvc_cnt * dfcombo.average_Medicare_allowed_amt

In [400]:
dfcombo['Net_submit_pay'] = dfcombo.Total_submitted_chrg_amt - dfcombo.Total_Medicare_std_payment_amt

In [401]:
dfcombo['Net_allow_pay'] = dfcombo['Total_Medicare_allowed_amt'] -  dfcombo['Total_Medicare_std_payment_amt']

In [402]:
dfcombo['ratio_pay/submit'] = dfcombo.Total_Medicare_std_payment_amt/ dfcombo.Total_submitted_chrg_amt

In [403]:
dfcombo['ratio_pay/allowed'] = dfcombo.Total_Medicare_std_payment_amt/ dfcombo.Total_Medicare_allowed_amt

In [404]:
print((dfcombo.line_srvc_cnt == dfcombo.bene_day_srvc_cnt).sum())
print(dfcombo.shape)
# drop bene_day_srvc_cnt since highly similar to line_srvc_cnt (prevent multi-collinearity)

1483371
(1511529, 31)


In [405]:
dfcombo.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,EXCLTYPE,year,Target,Total_Medicare_std_payment_amt,Total_Medicare_allowed_amt,Net_submit_pay,Net_allow_pay,ratio_pay/submit,ratio_pay/allowed
0,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,None,None,0,18243.00,23491.020000,87092.000000,5248.020000,0.173190,0.776595
1,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,None,None,0,35769.03,45534.280000,168786.970003,9765.250000,0.174862,0.785541
2,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,None,None,0,17221.58,22077.359999,78220.420000,4855.779999,0.180440,0.780056
3,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,None,None,0,6395.64,8070.120000,44004.360000,1674.480000,0.126898,0.792509
4,1003000142,khalil,rashid,M.D.,M,4126 n holland sylvania rd,toledo,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,None,None,0,6871.62,9149.970000,7189.380000,2278.350000,0.488701,0.750999


In [406]:
# DROP THESE COLUMNS!
dropcols = ["lastname", "firstname", "credentials", "address", "city", "zip",
         "place_of_service", "hcpcs_description","bene_day_srvc_cnt",
         'average_Medicare_payment_amt','Total_Medicare_payment_amt']

In [407]:
dfcombo = dfcombo.drop(dropcols, axis=1)

In [408]:
# Remove 10 states = Filter on state (total states = 51)
cols = ['XX','AA', 'AE','AP','AS','GU','MP','PR','VI','ZZ']
len(cols)

dfcombo = dfcombo[~dfcombo.state.isin(cols)]
dfcombo.state.nunique()

51

In [410]:
dfcombo[~dfcombo.state.isin(cols)].shape

(1505972, 20)

In [411]:
dfcombo.head()

,npi,gender,state,provider_type,hcpcs_code,line_srvc_cnt,bene_unique_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,EXCLTYPE,year,Target,Total_Medicare_std_payment_amt,Total_Medicare_allowed_amt,Net_submit_pay,Net_allow_pay,ratio_pay/submit,ratio_pay/allowed
0,1003000126,M,MD,Internal Medicine,99223,114.0,110,206.061579,923.991228,160.026316,105335.000000,None,None,0,18243.00,23491.020000,87092.000000,5248.020000,0.173190,0.776595
1,1003000126,M,MD,Internal Medicine,99232,627.0,233,72.622456,326.245614,57.047895,204556.000003,None,None,0,35769.03,45534.280000,168786.970003,9765.250000,0.174862,0.785541
2,1003000126,M,MD,Internal Medicine,99233,207.0,127,106.653913,461.072464,83.196039,95442.000000,None,None,0,17221.58,22077.359999,78220.420000,4855.779999,0.180440,0.780056
3,1003000126,M,MD,Internal Medicine,99291,36.0,23,224.170000,1400.000000,177.656667,50400.000000,None,None,0,6395.64,8070.120000,44004.360000,1674.480000,0.126898,0.792509
4,1003000142,M,OH,Anesthesiology,99213,129.0,89,70.930000,109.000000,53.268372,14061.000000,None,None,0,6871.62,9149.970000,7189.380000,2278.350000,0.488701,0.750999


### add weight factor (imbalanced dataset)

In [412]:
# Excluded unique NPI
ex_u = dfcombo[dfcombo.Target==1]['npi'].nunique()
ex_u

140

In [413]:
# Non-Excluded unique NPI
nex_u = dfcombo[dfcombo.Target==0]['npi'].nunique()
nex_u

648637

In [414]:
# Total unique NPI for Excluded + Non-Excluded
tot = ex_u + nex_u
tot

648777

In [415]:
# Find the weights: (1 / probability)
print("Weight for Target==1:", 1 / (ex_u / tot))
print("Weight for Target==0:", 1 / (nex_u / tot))

Weight for Target==1: 4634.121428571429
Weight for Target==0: 1.0002158372094099


In [416]:
# Make a new weights column:
dfcombo['weight'] = np.where(dfcombo.Target==1, 1/(ex_u/tot), 1/(nex_u/tot))

In [417]:
# weights value_counts() per npi
dfcombo.weight.value_counts()

1.000216       1505687
4634.121429        285
Name: weight, dtype: int64

In [418]:
# weights value_counts() per UNIQUE npi
dfcombo[~dfcombo.npi.duplicated('first')]['weight'].value_counts()

1.000216       648637
4634.121429       140
Name: weight, dtype: int64

In [419]:
dfcombo.columns

Index(['npi', 'gender', 'state', 'provider_type', 'hcpcs_code',
       'line_srvc_cnt', 'bene_unique_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_standard_amt',
       'Total_submitted_chrg_amt', 'EXCLTYPE', 'year', 'Target',
       'Total_Medicare_std_payment_amt', 'Total_Medicare_allowed_amt',
       'Net_submit_pay', 'Net_allow_pay', 'ratio_pay/submit',
       'ratio_pay/allowed', 'weight'],
      dtype='object')

### wide-form df per unique-NPI

In [420]:
valuecols = ['line_srvc_cnt', 'bene_unique_cnt', 
       'average_Medicare_allowed_amt', 'average_submitted_chrg_amt', 'average_Medicare_standard_amt',
       'Total_Medicare_allowed_amt', 'Total_submitted_chrg_amt', 'Total_Medicare_std_payment_amt', 
       'Net_submit_pay', 'Net_allow_pay', 
       'ratio_pay/submit', 'ratio_pay/allowed']

In [421]:
dfcombo_wide = dfcombo.pivot_table(index='npi', columns='hcpcs_code', values= valuecols)

In [422]:
dfcombo_wide.head(10)

Net_allow_pay                                                     \
hcpcs_code         66984    99213      99214    99223    99232        99233   
npi                                                                           
1003000126           NaN      NaN        NaN  5248.02  9765.25  4855.779999   
1003000142           NaN  2278.35   4497.905      NaN      NaN          NaN   
1003000407           NaN      NaN        NaN  2897.41  3512.85  1602.480000   
1003000423           NaN   382.41        NaN      NaN      NaN          NaN   
1003000480           NaN   399.93        NaN      NaN      NaN          NaN   
1003000522           NaN  8636.97  18682.550  1626.44  1344.01   395.930000   
1003000530           NaN  3458.26  26501.800      NaN      NaN          NaN   
1003000597           NaN  4606.05   6500.450      NaN   236.16          NaN   
1003000738           NaN   460.50        NaN      NaN      NaN          NaN   
1003000902           NaN  9860.38   5122.940      NaN      NaN          NaN   

                                      Net_submit_pay                        \
hcpcs_code 99284 99285    99291 A0427          66984     99213       99214   
npi                                                                          
1003000126   NaN   NaN  1674.48   NaN            NaN       NaN         NaN   
1003000142   NaN   NaN      NaN   NaN            NaN   7189.38   13931.375   
1003000407   NaN   NaN      NaN   NaN            NaN       NaN         NaN   
1003000423   NaN   NaN      NaN   NaN            NaN    782.88         NaN   
1003000480   NaN   NaN      NaN   NaN            NaN   4113.43         NaN   
1003000522   NaN   NaN      NaN   NaN            NaN  61135.77  125194.620   
1003000530   NaN   NaN      NaN   NaN            NaN   8109.62   62477.320   
1003000597   NaN   NaN      NaN   NaN            NaN  30410.85   37526.300   
1003000738   NaN   NaN      NaN   NaN            NaN   1877.25         NaN   
1003000902   NaN   NaN      NaN   NaN            NaN  41665.18   20647.740   

                                                                           \
hcpcs_code     99223          99232     99233 99284 99285     99291 A0427   
npi                                                                         
1003000126  87092.00  168786.970003  78220.42   NaN   NaN  44004.36   NaN   
1003000142       NaN            NaN       NaN   NaN   NaN       NaN   NaN   
1003000407   6068.08   22334.950000   7000.76   NaN   NaN       NaN   NaN   
1003000423       NaN            NaN       NaN   NaN   NaN       NaN   NaN   
1003000480       NaN            NaN       NaN   NaN   NaN       NaN   NaN   
1003000522  13200.24   13593.150000   4012.00   NaN   NaN       NaN   NaN   
1003000530       NaN            NaN       NaN   NaN   NaN       NaN   NaN   
1003000597       NaN    1756.800000       NaN   NaN   NaN       NaN   NaN   
1003000738       NaN            NaN       NaN   NaN   NaN       NaN   NaN   
1003000902       NaN            NaN       NaN   NaN   NaN       NaN   NaN   

           Total_Medicare_allowed_amt                                          \
hcpcs_code                      66984     99213     99214     99223     99232   
npi                                                                             
1003000126                        NaN       NaN       NaN  23491.02  45534.28   
1003000142                        NaN   9149.97  18181.53       NaN       NaN   
1003000407                        NaN       NaN       NaN  14840.33  17327.90   
1003000423                        NaN   1489.53       NaN       NaN       NaN   
1003000480                        NaN   1301.50       NaN       NaN       NaN   
1003000522                        NaN  26341.20  53521.93   5786.20   6093.86   
1003000530                        NaN   8788.64  69374.48       NaN       NaN   
1003000597                        NaN  22195.20  26094.15       NaN   1269.36   
1003000738                        NaN   1558.25       NaN       NaN       N

In [423]:
def imputezeros(df_input):
    df = df_input.copy()
    for col in df.columns:
        df[col] = np.where(df[col].isnull(), int(0), df[col])
    return(df)

In [424]:
dfcombo_wide = imputezeros(dfcombo_wide)
dfcombo_wide.head(10)

Net_allow_pay                                                     \
hcpcs_code         66984    99213      99214    99223    99232        99233   
npi                                                                           
1003000126           0.0     0.00      0.000  5248.02  9765.25  4855.779999   
1003000142           0.0  2278.35   4497.905     0.00     0.00     0.000000   
1003000407           0.0     0.00      0.000  2897.41  3512.85  1602.480000   
1003000423           0.0   382.41      0.000     0.00     0.00     0.000000   
1003000480           0.0   399.93      0.000     0.00     0.00     0.000000   
1003000522           0.0  8636.97  18682.550  1626.44  1344.01   395.930000   
1003000530           0.0  3458.26  26501.800     0.00     0.00     0.000000   
1003000597           0.0  4606.05   6500.450     0.00   236.16     0.000000   
1003000738           0.0   460.50      0.000     0.00     0.00     0.000000   
1003000902           0.0  9860.38   5122.940     0.00     0.00     0.000000   

                                      Net_submit_pay                        \
hcpcs_code 99284 99285    99291 A0427          66984     99213       99214   
npi                                                                          
1003000126   0.0   0.0  1674.48   0.0            0.0      0.00       0.000   
1003000142   0.0   0.0     0.00   0.0            0.0   7189.38   13931.375   
1003000407   0.0   0.0     0.00   0.0            0.0      0.00       0.000   
1003000423   0.0   0.0     0.00   0.0            0.0    782.88       0.000   
1003000480   0.0   0.0     0.00   0.0            0.0   4113.43       0.000   
1003000522   0.0   0.0     0.00   0.0            0.0  61135.77  125194.620   
1003000530   0.0   0.0     0.00   0.0            0.0   8109.62   62477.320   
1003000597   0.0   0.0     0.00   0.0            0.0  30410.85   37526.300   
1003000738   0.0   0.0     0.00   0.0            0.0   1877.25       0.000   
1003000902   0.0   0.0     0.00   0.0            0.0  41665.18   20647.740   

                                                                           \
hcpcs_code     99223          99232     99233 99284 99285     99291 A0427   
npi                                                                         
1003000126  87092.00  168786.970003  78220.42   0.0   0.0  44004.36   0.0   
1003000142      0.00       0.000000      0.00   0.0   0.0      0.00   0.0   
1003000407   6068.08   22334.950000   7000.76   0.0   0.0      0.00   0.0   
1003000423      0.00       0.000000      0.00   0.0   0.0      0.00   0.0   
1003000480      0.00       0.000000      0.00   0.0   0.0      0.00   0.0   
1003000522  13200.24   13593.150000   4012.00   0.0   0.0      0.00   0.0   
1003000530      0.00       0.000000      0.00   0.0   0.0      0.00   0.0   
1003000597      0.00    1756.800000      0.00   0.0   0.0      0.00   0.0   
1003000738      0.00       0.000000      0.00   0.0   0.0      0.00   0.0   
1003000902      0.00       0.000000      0.00   0.0   0.0      0.00   0.0   

           Total_Medicare_allowed_amt                                          \
hcpcs_code                      66984     99213     99214     99223     99232   
npi                                                                             
1003000126                        0.0      0.00      0.00  23491.02  45534.28   
1003000142                        0.0   9149.97  18181.53      0.00      0.00   
1003000407                        0.0      0.00      0.00  14840.33  17327.90   
1003000423                        0.0   1489.53      0.00      0.00      0.00   
1003000480                        0.0   1301.50      0.00      0.00      0.00   
1003000522                        0.0  26341.20  53521.93   5786.20   6093.86   
1003000530                        0.0   8788.64  69374.48      0.00      0.00   
1003000597                        0.0  22195.20  26094.15      0.00   1269.36   
1003000738                        0.0   1558.25      0.00      0.00      0.

In [425]:
# flattening the multi-level index column names:
dfcombo_wide.columns = list(map('_'.join , dfcombo_wide.columns.values))

In [382]:
dfcombo_wide.head(10)

,Net_allow_pay_66984,Net_allow_pay_99213,Net_allow_pay_99214,Net_allow_pay_99223,Net_allow_pay_99232,Net_allow_pay_99233,Net_allow_pay_99284,Net_allow_pay_99285,Net_allow_pay_99291,Net_allow_pay_A0427,Net_submit_pay_66984,Net_submit_pay_99213,Net_submit_pay_99214,Net_submit_pay_99223,Net_submit_pay_99232,Net_submit_pay_99233,Net_submit_pay_99284,Net_submit_pay_99285,Net_submit_pay_99291,Net_submit_pay_A0427,Total_Medicare_allowed_amt_66984,Total_Medicare_allowed_amt_99213,Total_Medicare_allowed_amt_99214,Total_Medicare_allowed_amt_99223,Total_Medicare_allowed_amt_99232,Total_Medicare_allowed_amt_99233,Total_Medicare_allowed_amt_99284,Total_Medicare_allowed_amt_99285,Total_Medicare_allowed_amt_99291,Total_Medicare_allowed_amt_A0427,Total_Medicare_std_payment_amt_66984,Total_Medicare_std_payment_amt_99213,Total_Medicare_std_payment_amt_99214,Total_Medicare_std_payment_amt_99223,Total_Medicare_std_payment_amt_99232,Total_Medicare_std_payment_amt_99233,Total_Medicare_std_payment_amt_99284,Total_Medicare_std_payment_amt_99285,Total_Medicare_std_payment_amt_99291,Total_Medicare_std_payment_amt_A0427,Total_submitted_chrg_amt_66984,Total_submitted_chrg_amt_99213,Total_submitted_chrg_amt_99214,Total_submitted_chrg_amt_99223,Total_submitted_chrg_amt_99232,Total_submitted_chrg_amt_99233,Total_submitted_chrg_amt_99284,Total_submitted_chrg_amt_99285,Total_submitted_chrg_amt_99291,Total_submitted_chrg_amt_A0427,average_Medicare_allowed_amt_66984,average_Medicare_allowed_amt_99213,average_Medicare_allowed_amt_99214,average_Medicare_allowed_amt_99223,average_Medicare_allowed_amt_99232,average_Medicare_allowed_amt_99233,average_Medicare_allowed_amt_99284,average_Medicare_allowed_amt_99285,average_Medicare_allowed_amt_99291,average_Medicare_allowed_amt_A0427,average_Medicare_standard_amt_66984,average_Medicare_standard_amt_99213,average_Medicare_standard_amt_99214,average_Medicare_standard_amt_99223,average_Medicare_standard_amt_99232,average_Medicare_standard_amt_99233,average_Medicare_standard_amt_99284,average_Medicare_standard_amt_99285,average_Medicare_standard_amt_99291,average_Medicare_standard_amt_A0427,average_submitted_chrg_amt_66984,average_submitted_chrg_amt_99213,average_submitted_chrg_amt_99214,average_submitted_chrg_amt_99223,average_submitted_chrg_amt_99232,average_submitted_chrg_amt_99233,average_submitted_chrg_amt_99284,average_submitted_chrg_amt_99285,average_submitted_chrg_amt_99291,average_submitted_chrg_amt_A0427,bene_unique_cnt_66984,bene_unique_cnt_99213,bene_unique_cnt_99214,bene_unique_cnt_99223,bene_unique_cnt_99232,bene_unique_cnt_99233,bene_unique_cnt_99284,bene_unique_cnt_99285,bene_unique_cnt_99291,bene_unique_cnt_A0427,line_srvc_cnt_66984,line_srvc_cnt_99213,line_srvc_cnt_99214,line_srvc_cnt_99223,line_srvc_cnt_99232,line_srvc_cnt_99233,line_srvc_cnt_99284,line_srvc_cnt_99285,line_srvc_cnt_99291,line_srvc_cnt_A0427,ratio_pay/allowed_66984,ratio_pay/allowed_99213,ratio_pay/allowed_99214,ratio_pay/allowed_99223,ratio_pay/allowed_99232,ratio_pay/allowed_99233,ratio_pay/allowed_99284,ratio_pay/allowed_99285,ratio_pay/allowed_99291,ratio_pay/allowed_A0427,ratio_pay/submit_66984,ratio_pay/submit_99213,ratio_pay/submit_99214,ratio_pay/submit_99223,ratio_pay/submit_99232,ratio_pay/submit_99233,ratio_pay/submit_99284,ratio_pay/submit_99285,ratio_pay/submit_99291,ratio_pay/submit_A0427,gender,state,provider_type,EXCLTYPE,year,Target,weight
npi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1003000126,0.0,0.00,0.000,5248.02,9765.25,4855.779999,0.0,0.0,1674.48,0.0,0.0,0.00,0.000,87092.00,168786.970003,78220.42,0.0,0.0,44004.36,0.0,0.0,0.00,0.00,23491.02,45534.28,22077.359999,0.0,0.0,8070.12,0.0,0.0,0.00,0.000,18243.00,35769.03,17221.58,0.0,0.0,6395.64,0.0,0.0,0.0,0.0,105335.0,204556.000003,95442.0,0.0,0.0,50400.0,0.0,0.0,0.00,0.000,206.061579,72.622456,106.653913,0.0,0.0,224.17,0.0,0.0,0.000000,0.000000,160.026316,57.047895,83.196039,0.0,0.0,177.

In [427]:
dfcombo_wide.shape

(648777, 120)

In [428]:
# Add 8 columns per unique npi
keep = ['npi','gender', 'state', 'provider_type', 'EXCLTYPE', 'year', 'Target', 'weight']
dfadd = dfcombo.filter(items=keep, axis=1)
dfadd = dfadd.drop_duplicates()
dfadd.shape

(648778, 8)

In [430]:
df_final = pd.merge(dfcombo_wide, dfadd, how='left', on='npi')
df_final.head(10)

,npi,Net_allow_pay_66984,Net_allow_pay_99213,Net_allow_pay_99214,Net_allow_pay_99223,Net_allow_pay_99232,Net_allow_pay_99233,Net_allow_pay_99284,Net_allow_pay_99285,Net_allow_pay_99291,Net_allow_pay_A0427,Net_submit_pay_66984,Net_submit_pay_99213,Net_submit_pay_99214,Net_submit_pay_99223,Net_submit_pay_99232,Net_submit_pay_99233,Net_submit_pay_99284,Net_submit_pay_99285,Net_submit_pay_99291,Net_submit_pay_A0427,Total_Medicare_allowed_amt_66984,Total_Medicare_allowed_amt_99213,Total_Medicare_allowed_amt_99214,Total_Medicare_allowed_amt_99223,Total_Medicare_allowed_amt_99232,Total_Medicare_allowed_amt_99233,Total_Medicare_allowed_amt_99284,Total_Medicare_allowed_amt_99285,Total_Medicare_allowed_amt_99291,Total_Medicare_allowed_amt_A0427,Total_Medicare_std_payment_amt_66984,Total_Medicare_std_payment_amt_99213,Total_Medicare_std_payment_amt_99214,Total_Medicare_std_payment_amt_99223,Total_Medicare_std_payment_amt_99232,Total_Medicare_std_payment_amt_99233,Total_Medicare_std_payment_amt_99284,Total_Medicare_std_payment_amt_99285,Total_Medicare_std_payment_amt_99291,Total_Medicare_std_payment_amt_A0427,Total_submitted_chrg_amt_66984,Total_submitted_chrg_amt_99213,Total_submitted_chrg_amt_99214,Total_submitted_chrg_amt_99223,Total_submitted_chrg_amt_99232,Total_submitted_chrg_amt_99233,Total_submitted_chrg_amt_99284,Total_submitted_chrg_amt_99285,Total_submitted_chrg_amt_99291,Total_submitted_chrg_amt_A0427,average_Medicare_allowed_amt_66984,average_Medicare_allowed_amt_99213,average_Medicare_allowed_amt_99214,average_Medicare_allowed_amt_99223,average_Medicare_allowed_amt_99232,average_Medicare_allowed_amt_99233,average_Medicare_allowed_amt_99284,average_Medicare_allowed_amt_99285,average_Medicare_allowed_amt_99291,average_Medicare_allowed_amt_A0427,average_Medicare_standard_amt_66984,average_Medicare_standard_amt_99213,average_Medicare_standard_amt_99214,average_Medicare_standard_amt_99223,average_Medicare_standard_amt_99232,average_Medicare_standard_amt_99233,average_Medicare_standard_amt_99284,average_Medicare_standard_amt_99285,average_Medicare_standard_amt_99291,average_Medicare_standard_amt_A0427,average_submitted_chrg_amt_66984,average_submitted_chrg_amt_99213,average_submitted_chrg_amt_99214,average_submitted_chrg_amt_99223,average_submitted_chrg_amt_99232,average_submitted_chrg_amt_99233,average_submitted_chrg_amt_99284,average_submitted_chrg_amt_99285,average_submitted_chrg_amt_99291,average_submitted_chrg_amt_A0427,bene_unique_cnt_66984,bene_unique_cnt_99213,bene_unique_cnt_99214,bene_unique_cnt_99223,bene_unique_cnt_99232,bene_unique_cnt_99233,bene_unique_cnt_99284,bene_unique_cnt_99285,bene_unique_cnt_99291,bene_unique_cnt_A0427,line_srvc_cnt_66984,line_srvc_cnt_99213,line_srvc_cnt_99214,line_srvc_cnt_99223,line_srvc_cnt_99232,line_srvc_cnt_99233,line_srvc_cnt_99284,line_srvc_cnt_99285,line_srvc_cnt_99291,line_srvc_cnt_A0427,ratio_pay/allowed_66984,ratio_pay/allowed_99213,ratio_pay/allowed_99214,ratio_pay/allowed_99223,ratio_pay/allowed_99232,ratio_pay/allowed_99233,ratio_pay/allowed_99284,ratio_pay/allowed_99285,ratio_pay/allowed_99291,ratio_pay/allowed_A0427,ratio_pay/submit_66984,ratio_pay/submit_99213,ratio_pay/submit_99214,ratio_pay/submit_99223,ratio_pay/submit_99232,ratio_pay/submit_99233,ratio_pay/submit_99284,ratio_pay/submit_99285,ratio_pay/submit_99291,ratio_pay/submit_A0427,gender,state,provider_type,EXCLTYPE,year,Target,weight
0,1003000126,0.0,0.00,0.000,5248.02,9765.25,4855.779999,0.0,0.0,1674.48,0.0,0.0,0.00,0.000,87092.00,168786.970003,78220.42,0.0,0.0,44004.36,0.0,0.0,0.00,0.00,23491.02,45534.28,22077.359999,0.0,0.0,8070.12,0.0,0.0,0.00,0.000,18243.00,35769.03,17221.58,0.0,0.0,6395.64,0.0,0.0,0.0,0.0,105335.0,204556.000003,95442.0,0.0,0.0,50400.0,0.0,0.0,0.00,0.000,206.061579,72.622456,106.653913,0.0,0.0,224.17,0.0,0.0,0.000000,0.000000,160.026316,57.047895,83.196039,0.0,0.0,177.656667,0.0,0.0,0.0,0.0,923.991228,326.245614,461.072464,0.0,0.0,1400.0,0.0,0.0,0.0,0.0,110.0,233.0,127.0,0.0,0.0,23.0,0.0,0.0

In [432]:
len(df_final.columns)

128

In [434]:
df_final.shape

(648778, 128)

In [433]:
#df_final.to_csv("data/widedf.csv",index_label=False) #writes to csv